In [1]:
from pymilvus import MilvusClient
from llm import StarEncoder, CodeBERT
import torch
from llm2vec import LLM2Vec
from huggingface_hub import login
from datasets import load_dataset
from tqdm import tqdm
import numpy as np
from evaluate import evaluate

# Database connection and setup

In [2]:
client = MilvusClient(uri="http://localhost:19530", user="root", password="Milvus")

In [3]:
models = ["bert", "starcoder"]
for current_model in models:
    collection_name = current_model

    client.create_collection(
        collection_name=collection_name,
        dimension=768,
        auto_id=True,
    )
client.create_collection(
    collection_name="llama",
    dimension=2048,
    auto_id=True,
)

# Refresh databases

In [5]:
models = ["bert", "starcoder"]
for current_model in models:
    collection_name = current_model
    client.drop_collection(
        collection_name=collection_name,
    )
    client.create_collection(
        collection_name=collection_name,
        dimension=768,
        auto_id=True,
    )

client.drop_collection(
    collection_name=collection_name,
)
client.create_collection(
    collection_name="llama",
    dimension=2048,
    auto_id=True,
)

# Evaluate

In [5]:
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

EVAL_CONFIGS = {
    "starencoder": {
        "max_input_len": 10000,
        "maximum_token_len": 1024,
        "device": DEVICE,
    },
    "codebert": {
        "max_input_len": 10000,
        "maximum_token_len": 512,
        "device": DEVICE,
    },
}

In [8]:
dataset = load_dataset("C:\\CSN")

In [33]:
bert = CodeBERT(**EVAL_CONFIGS.get("codebert"))

In [36]:
evaluate(bert, client, dataset, "bert")
print("done")

  4%|▎         | 1000/26909 [12:47<5:31:26,  1.30it/s]

done


In [6]:
starencoder = StarEncoder(**EVAL_CONFIGS.get("starencoder"))

# Enter CSN-Dataset into Milvus

In [10]:
evaluate(starencoder, client, dataset, "starcoder")
print("done")

  0%|          | 0/26909 [00:00<?, ?it/s]

  3%|▎         | 782/26909 [05:28<3:41:47,  1.96it/s] 

In [364]:
from llm2vec import LLM2Vec

import torch
from transformers import AutoTokenizer, AutoModel, AutoConfig
from peft import PeftModel

# Loading base Mistral model, along with custom code that enables bidirectional connections in decoder-only LLMs. MNTP LoRA weights are merged into the base model.
tokenizer = AutoTokenizer.from_pretrained("McGill-NLP/LLM2Vec-Sheared-LLaMA-mntp")
config = AutoConfig.from_pretrained(
    "McGill-NLP/LLM2Vec-Sheared-LLaMA-mntp", trust_remote_code=True
)
llama = AutoModel.from_pretrained(
    "McGill-NLP/LLM2Vec-Sheared-LLaMA-mntp",
    trust_remote_code=True,
    config=config,
    torch_dtype=torch.bfloat16,
    device_map="cuda" if torch.cuda.is_available() else "cpu",
)
llama = PeftModel.from_pretrained(
    llama,
    "McGill-NLP/LLM2Vec-Sheared-LLaMA-mntp",
)
llama = llama.merge_and_unload()  # This can take several minutes on cpu

# Loading supervised model. This loads the trained LoRA weights on top of MNTP model. Hence the final weights are -- Base model + MNTP (LoRA) + supervised (LoRA).
llama = PeftModel.from_pretrained(
    llama, "McGill-NLP/LLM2Vec-Sheared-LLaMA-mntp-supervised"
)

# Wrapper for encoding and pooling operations
l2v = LLM2Vec(llama, tokenizer, pooling_mode="mean", max_length=512)

tokenizer_config.json:   0%|          | 0.00/871 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/753 [00:00<?, ?B/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

d:\llm-utilities\venv\lib\site-packages\transformers\models\llama\modeling_llama.py:670: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

tensor([[0.6504, 0.1296],
        [0.0918, 0.4725]])


'\ntensor([[0.6500, 0.1291],\n        [0.0916, 0.4733]])\n'

In [390]:
evaluate(llama, client, dataset, "llama", normalize=True)
print("done")

  0%|          | 0/26909 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 1/26909 [00:00<4:18:18,  1.74it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 2/26909 [00:00<3:09:43,  2.36it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

  4%|▎         | 1000/26909 [05:45<2:09:48,  3.33it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

  4%|▎         | 1000/26909 [05:45<2:29:06,  2.90it/s]

done


In [397]:
from transformers.utils.logging import disable_progress_bar
import datasets
import os

from tqdm import tqdm
from functools import partialmethod

tqdm.__init__ = partialmethod(tqdm.__init__, disable=True)

os.environ["TQDM_DISABLE"] = "1"
disable_progress_bar()
datasets.disable_progress_bar()
l2v.encode("test")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

tensor([[-1.7109, -0.3945,  0.7812,  ...,  0.9297,  1.8594,  0.2021],
        [-1.7109,  0.4961,  1.6328,  ...,  1.1094, -0.2148, -0.5859],
        [-1.8438, -0.2061,  1.0234,  ...,  1.1094, -0.5156,  0.2539],
        [-1.7109, -0.3945,  0.7812,  ...,  0.9297,  1.8594,  0.2021]])

In [401]:
import transformers

transformers.logging.set_verbosity_error()

ranks = []
for idx, example in enumerate(tqdm(dataset["test"].shuffle(seed=42))):

    # print(example["func_code_string"])
    query_vector = l2v.encode([example["func_documentation_string"]])

    q_reps_norm = torch.nn.functional.normalize(query_vector, p=2, dim=1)

    res = client.search(
        collection_name="llama",  # target collection
        data=[q_reps_norm[0].numpy()],  # query vectors
        limit=1000,  # number of returned entities
        output_fields=["text"],  # specifies fields to be returned
        search_params={"metric_type": "COSINE"},
    )

    for index, result in enumerate(res[0]):
        if example["func_code_string"] == result["entity"]["text"]:
            ranks.append(1 / (index + 1))

    if idx == 1000:
        break
print("MRR = ", np.mean(ranks))
print("done")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

MRR =  0.8296693368434425
done


In [402]:
print(np.mean(ranks))

0.8296693368434425
